In [117]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [118]:
import os
import re
import pandas as pd

# Ruta de la carpeta que contiene los archivos de texto de las facturas
ruta_carpeta = "/content/drive/MyDrive/Texto de Facturas"

# Lista para almacenar los diccionarios de facturas
facturas = []

# Iterar sobre los archivos en la carpeta
for nombre_archivo in os.listdir(ruta_carpeta):
    # Construir la ruta completa del archivo
    ruta_archivo = os.path.join(ruta_carpeta, nombre_archivo)

    # Leer el contenido del archivo
    with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
        texto_factura = archivo.read()

    # Expresiones regulares para extraer datos
    numero_factura_match = re.search(r'Numero factura: N°\s*(\d+)', texto_factura)
    fecha_emision_match = re.search(r'Fecha Emision:\s*(.+)', texto_factura)
    senores_match = re.search(r'SEÑOR\(ES\):\s*(.+)', texto_factura)

    # Intentar ambas expresiones regulares para extraer el RUT
    rut_match1 = re.search(r'R\.U\.T\.: \s*(\d{1,2}\.\d{3}\.\d{3}-?\s?\d{1,2})', texto_factura)
    rut_match2 = re.search(r'R\.U\.T\.: \s*(\d{1,2}\.\d{3}\.\d{3}-\s?[Kk\d]{1,2})', texto_factura)
    rut_match = rut_match1 if rut_match1 else rut_match2

    descripcion_match = re.search(r'Descripcion:\s*(.+?)TOTAL:', texto_factura, re.DOTALL)
    total_match = re.search(r'TOTAL: \$\s*([\d,.]+)', texto_factura)

    # Verificar si se encontraron coincidencias y extraer datos
    if all(match is not None for match in [numero_factura_match, fecha_emision_match, senores_match, rut_match, descripcion_match, total_match]):
        numero_factura = numero_factura_match.group(1)
        fecha_emision = fecha_emision_match.group(1)
        senores = senores_match.group(1)
        rut = rut_match.group(1).replace(".", "").replace(" ", "")  # Eliminar puntos y espacios en el RUT
        descripcion = descripcion_match.group(1).strip()
        total = float(total_match.group(1).replace('.', '').replace(',', ''))  # Eliminar puntos y coma del total

        # Crear un diccionario con los datos
        datos_factura = {
            "Numero Factura": int(numero_factura),  # Convertir a entero para facilitar la ordenación
            "Fecha Emision": fecha_emision,
            "Señor(es)": senores,
            "RUT": rut,
            "Descripcion": descripcion,
            "Total": total
        }

        # Agregar el diccionario a la lista
        facturas.append(datos_factura)

# Crear un DataFrame de pandas
df_facturas = pd.DataFrame(facturas)

# Ordenar el DataFrame por el número de factura
df_facturas.sort_values(by='Numero Factura', inplace=True)

# Guardar el DataFrame en un archivo Excel
df_facturas.to_excel('/content/drive/MyDrive/facturas.xlsx', index=False)

# Imprimir los datos de las facturas
print(df_facturas)

     Numero Factura             Fecha Emision  \
8                 1    25 de Octubre del 2016   
215               3  15 de Noviembre del 2016   
202               4  07 de Diciembre del 2016   
207               5  20 de Diciembre del 2016   
191               6  23 de Diciembre del 2016   
..              ...                       ...   
223             277  30 de Noviembre del 2023   
101             278  15 de Diciembre del 2023   
224             279  21 de Diciembre del 2023   
222             280  29 de Diciembre del 2023   
236             281      18 de Enero del 2024   

                                          Señor(es)         RUT  \
8            CORP ADMINISTRATIVA DEL PODER JUDICIAL  60301001-9   
215          CORP ADMINISTRATIVA DEL PODER JUDICIAL  60301001-9   
202          CORP ADMINISTRATIVA DEL PODER JUDICIAL  60301001-9   
207          CORP ADMINISTRATIVA DEL PODER JUDICIAL  60301001-9   
191          CORP ADMINISTRATIVA DEL PODER JUDICIAL  60301001-9   
..       